### 1) 모델 세팅, 데이터 세팅, 함수 세팅

In [5]:
import re
import ntpath 
from BuildModel import BuildModel
from ModelType import ModelType
import torch

RANGES = [
    ((1, 50),   "Happiness"),
    ((51, 100), "Surprise"),
    ((101,150), "Neutral"),
    ((151,200), "Fear"),
    ((201,250), "Disgust"),
    ((251,300), "Anger"),
    ((301,350), "Sadness"),
]
CLASSES = list(map(lambda x: x[1], RANGES))
label2idx = {name:i  for i,name in enumerate(CLASSES)}

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BuildModel(ModelType.CONVNEXT_SMALL,len(RANGES)).model
model.to(device)


def score_to_label(score:int):
    for (low,high), label in RANGES:
        if low <= score <= high: 
            return label

def parse_label_from_name(path:str):
    stem = ntpath.splitext(ntpath.basename(path))[0]
    m = re.match(r'^\d+-(\d+)_mel$',stem)
    return int(m.group(1))

def parse_label_idx(path: str) -> int:
    score = parse_label_from_name(path)
    name  = score_to_label(score)
    return label2idx[name]

c:\PythonProject\aug-08month_project5\.venv\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


### 2) 이미지 전처리

In [6]:
from torchvision import transforms as T
from torchvision.models import ConvNeXt_Small_Weights
preprocess = ConvNeXt_Small_Weights.IMAGENET1K_V1.transforms()

# 멜스펙 1채널 -> 3채널
train_tf = T.Compose([
    T.Grayscale(3),  
    preprocess,                          
])
test_tf = T.Compose([
    T.Grayscale(3),
    preprocess,
])

### 3) 학습, 검증 데이터 분류

In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

ROOT = "C:\\PythonProject\\aug-08month_project5\\jin_sup\\mel_image"
all_full_paths = sorted([str(p) for p in Path(ROOT).rglob("*.png")])
y_all = list(map(parse_label_idx, all_full_paths))

X_train, X_test, y_train, y_test = train_test_split(
    all_full_paths, y_all, test_size=0.2, stratify=y_all, random_state=42
)

### 4) 배치 및 전처리 적용

In [11]:
from PIL import Image
class MelSpecImageDataset(Dataset):
    def __init__(self, paths, transform):
        self.paths = list(paths)
        self.transform = transform
    def __len__(self): return len(self.paths)
    def __getitem__(self, i):
        p = self.paths[i]
        y = parse_label_idx(p)
        img = Image.open(p).convert("RGB")
        x = self.transform(img)
        return x, y

In [12]:
X_train_ds = MelSpecImageDataset(X_train, train_tf)
X_test_ds = MelSpecImageDataset(X_test, test_tf)

BATCH_SIZE = 32
NUM_WORKERS = 4
X_train_dl = DataLoader(
    X_train_ds,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

X_test_dl = DataLoader(
    X_test_ds,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True,
)

### 5) 모델 학습 및 텐서 보드

In [16]:
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import torch.nn as nn
from torch import amp

criterion = nn.CrossEntropyLoss()
optimz = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)
import torch
print(torch.cuda.is_available())   # False 라고 나오면 GPU 안잡힘
print(torch.__version__)           # torch 버전 확인


False
2.8.0+cpu


In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

writer = SummaryWriter(log_dir="runs/convnext_experiment1")

EPOCHS = 10
scaler = amp.GradScaler("cuda")

step = 0
for epoch in range(1, EPOCHS + 1):
    model.train()
    
    for train, label in tqdm(X_test_dl):
        optimz.zero_grad()
        train, label = train.to(device), label.to(device)
        with amp.autocast("cuda"):
            pred = model(train)
            loss = criterion(pred, label)
        
        scaler.scale(loss).backward()
        scaler.step(optimz)
        scaler.update()
        
        #텐서 보드 기록
        writer.add_scalar("Loss/train", loss.item(), global_step)
        global_step += 1


c:\PythonProject\aug-08month_project5\.venv\lib\site-packages\torch\amp\grad_scaler.py:136: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
  0%|          | 0/65 [00:00<?, ?it/s]c:\PythonProject\aug-08month_project5\.venv\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


### 6) 모델 평가 및 저장

In [ ]:
from datetime import datetime
import os

y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for train, label in y_test:
        train, label = train.to(device), label.to(device)
        pred = model.argmax(1)
        
        y_true.extend(label.cpu().tolist())
        y_pred.extend(pred.cpu().tolist())

f1_macro = f1_score(y_true, y_pred, average="macro")   # 클래스별 f1을 평균
f1_micro = f1_score(y_true, y_pred, average="micro")   # 전체 샘플 기준 평균
f1_weighted = f1_score(y_true, y_pred, average="weighted")  # 클래스 비율 고려

print("F1(macro):", f1_macro)
print("F1(micro):", f1_micro)
print("F1(weighted):", f1_weighted)

print(classification_report(y_true, y_pred, target_names=CLASSES, digits=4))

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
filename = f"result_{timestamp}"
torch.save(model.state_dict(), f"model_{filename}.pth")